In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score

In [4]:
import numpy as np
import random
import os

seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)


In [5]:
# 학습 데이터
train_data = [
    ("I love this movie", 1),
    ("This film was terrible", 0),
    ("Absolutely fantastic!", 1),
    ("I did not like this movie", 0),
    ("What a great experience", 1),
    ("The acting was horrible", 0),
    ("I enjoyed every moment", 1),
    ("The plot was boring", 0),
    ("Brilliant performance by the actors", 1),
    ("Worst movie I have ever seen", 0),
    ("The soundtrack was amazing", 1),
    ("I will never watch this again", 0),
]

# 테스트 데이터 (훈련에 없던 단어도 일부 포함)
test_data = [
    ("This movie is amazing", 1),
    ("I hated the film", 0),
    ("The story was wonderful", 1),
    ("It was a waste of time", 0),
    ("Such an excellent movie", 1),
    ("The characters were disappointing", 0),
]


In [ ]:
# 토큰화 및 정수 인코딩
# Tokenizer는 문장을 단어 단위로 나누고, 각 단어를 고유 숫자로 매핑합니다.
# oov_token="<OOV>"는 학습되지 않은(처음 보는) 단어를 위한 기본 토큰입니다.
tokenizer = Tokenizer(oov_token="<OOV")
tokenizer.fit_on_texts([text for text, _ in train_data]) # 학습 데이터에서 어휘집 생성
vocab_size = len(tokenizer.word_index) + 1  # 어휘집 크기 (패딩 인덱스 용도를 위해 +1을 해줘야 함)

In [7]:
# train + test 전체 문장을 합쳐서 최대 길이 찾기
all_texts = [t for t, _ in train_data + test_data]
max_len = max(len(seq) for seq in tokenizer.texts_to_sequences(all_texts))

# 텍스트를 정수 인덱스로 변환 후, 시퀀스 길이를 맞춰주는 함수입니다.
# pad_sequences는 모든 문장의 길이를 동일하게 맞추기 위해 짧은 문장을 패딩합니다.
def encode_and_pad(data, maxlen):
    texts, labels = zip(*data)
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, maxlen=maxlen, padding='post')
    return np.array(padded), np.array(labels)


In [8]:
# 학습 데이터 및 테스트 데이터를 전처리합니다.
X_train, y_train = encode_and_pad(train_data, max_len)
X_test, y_test = encode_and_pad(test_data, max_len)

In [9]:
# 하이퍼파라미터 설정
embedding_dim = 100  # 각 단어를 100차원 벡터로 표현
hidden_dim = 128     # RNN의 은닉 상태 크기
output_dim = 2       # 출력 클래스 수 (긍정 vs 부정)

In [12]:
# RNN 모델 정의
model = Sequential([
    # 임베딩 레이어: 각 단어를 고정된 차원의 벡터로 변환합니다.
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=X_train.shape[1]),
    
    # RNN 레이어: 순차적으로 들어오는 단어 벡터를 기반으로 문장의 흐름을 학습합니다.
    SimpleRNN(units=hidden_dim),
    
    # 출력층: 2개의 노드 (긍정/부정)를 softmax 함수로 확률 출력
    Dense(output_dim, activation='softmax')
])

In [ ]:
# 모델 컴파일 및 학습
# 손실 함수는 sparse_categorical_crossentropy: 정수형 레이블을 사용할 때 적합합니다.
# 옵티마이저는 Adam으로 빠른 수렴을 기대할 수 있습니다.
model.compile(optimizer='adam',
        loss ='sparse_categorical_crossentropy',
        metrics=['accuracy'])

# fit 함수를 통해 모델을 학습합니다.
# verbose=1은 학습 과정을 상세히 출력합니다.
model.fit(X_train, y_train, epochs=5, verbose=1)

Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.5000 - loss: 0.7141
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - accuracy: 0.8333 - loss: 0.6431
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step - accuracy: 0.9167 - loss: 0.5783
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - accuracy: 1.0000 - loss: 0.5147
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step - accuracy: 1.0000 - loss: 0.4502


In [15]:
# 모델 평가
# predict 함수를 사용해 테스트 데이터를 예측합니다.
# np.argmax를 이용해 softmax 결과에서 가장 높은 확률의 클래스를 선택합니다.
y_pred = np.argmax(model.predict(X_test), axis=1)

# sklearn의 accuracy_score를 사용해 정확도를 측정합니다.
acc = accuracy_score(y_test, y_pred)
print(f'\nTest Accuracy: {acc:.3f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step

Test Accuracy: 0.833


In [ ]:
"""
성능 불일치 설명

이 코드에서 loss는 줄어들지만, accuracy는 일관되지 않게 변할 수 있습니다.
그 이유는:
1. 작은 데이터셋 크기: 모델이 다양한 패턴을 학습하지 못합니다.
2. 가중치 초기화: RNN 가중치가 매번 무작위로 초기화되어 결과가 다를 수 있습니다.
3. 과적합: 모델이 훈련 데이터에 너무 맞춰져, 테스트 성능이 저하될 수 있습니다.

이를 해결하려면 데이터셋을 확장하거나 정규화 기법을 적용할 수 있습니다.
"""